<a href="https://colab.research.google.com/github/sheon1206/Weekly_Reading_Review/blob/main/%EB%B9%84%EB%AC%B8%ED%95%99%EB%8F%85%ED%95%B4_%EC%A3%BC%EA%B0%84%ED%85%8C%EC%8A%A4%ED%8A%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# [1] 라이브러리 설치 및 폰트 설정
# reportlab 설치
!pip install reportlab

import os
import urllib.request
from reportlab.pdfbase import pdfmetrics
from reportlab.pdfbase.ttfonts import TTFont
from reportlab.pdfbase.pdfmetrics import registerFontFamily

def setup_fonts():
    print("폰트 설정을 시작합니다...")

    # 나눔고딕 폰트 다운로드
    font_urls = {
        'NanumGothic.ttf': 'https://github.com/google/fonts/raw/main/ofl/nanumgothic/NanumGothic-Regular.ttf',
        'NanumGothicBold.ttf': 'https://github.com/google/fonts/raw/main/ofl/nanumgothic/NanumGothic-Bold.ttf'
    }

    for font_name, url in font_urls.items():
        if not os.path.exists(font_name):
            print(f"다운로드 중: {font_name}...")
            try:
                urllib.request.urlretrieve(url, font_name)
            except Exception as e:
                print(f"다운로드 실패 ({font_name}): {e}")
                return False

    try:
        # 폰트 등록
        pdfmetrics.registerFont(TTFont('NanumGothic', 'NanumGothic.ttf'))
        pdfmetrics.registerFont(TTFont('NanumGothicBold', 'NanumGothicBold.ttf'))
        registerFontFamily('NanumGothic', normal='NanumGothic', bold='NanumGothicBold')
        print("성공! 폰트 등록이 완료되었습니다.")
        return True
    except Exception as e:
        print(f"폰트 등록 중 에러 발생: {e}")
        return False

# 실행
setup_fonts()

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 15.5 MB/s eta 0:00:00
폰트 설정을 시작합니다...
다운로드 중: NanumGothic.ttf...
다운로드 중: NanumGothicBold.ttf...
성공! 폰트 등록이 완료되었습니다.


True

In [2]:
import json, os, re, ast
from google.colab import drive
from reportlab.lib.pagesizes import A4
from reportlab.lib import colors
from reportlab.lib.styles import getSampleStyleSheet, ParagraphStyle
from reportlab.lib.units import mm
from reportlab.platypus import (
    BaseDocTemplate, Frame, PageTemplate, Paragraph, Spacer,
    Table, TableStyle, PageBreak, FrameBreak, NextPageTemplate,
    KeepTogether, HRFlowable
)
from reportlab.lib.enums import TA_JUSTIFY, TA_CENTER, TA_LEFT
from reportlab.pdfbase import pdfmetrics
from reportlab.pdfbase.ttfonts import TTFont

# -----------------------------------------------------------
# [1] 설정 및 준비
# -----------------------------------------------------------
drive.mount('/content/drive')

BASE_DIR = '/content/drive/MyDrive/weekly_kreading_test_data'
if not os.path.exists(BASE_DIR):
    os.makedirs(BASE_DIR)

INPUT_FILE = os.path.join(BASE_DIR, 'kreading_test_data.json')
OUTPUT_FILE = os.path.join(BASE_DIR, 'Weekly_KReading_Review.pdf')

try:
    pdfmetrics.registerFont(TTFont('NanumGothic', 'NanumGothic.ttf'))
    pdfmetrics.registerFont(TTFont('NanumGothicBold', 'NanumGothicBold.ttf'))
except:
    pass

# -----------------------------------------------------------
# [2] 스타일 설정
# -----------------------------------------------------------
def get_custom_styles():
    s = getSampleStyleSheet()
    try: s['Normal'].fontName = 'NanumGothic'
    except: pass

    s['Normal'].fontSize = 10
    s['Normal'].leading = 14.5
    s['Normal'].alignment = TA_JUSTIFY

    def add_style(name, parent, **kwargs):
        kwargs.setdefault('fontName', 'NanumGothic')
        s.add(ParagraphStyle(name=name, parent=s[parent], **kwargs))

    # [수정] MainTitle spaceAfter 축소 (20 -> 10)
    add_style('MainTitle', 'Heading1', fontName='NanumGothicBold', fontSize=20, alignment=TA_CENTER, spaceAfter=10)
    add_style('PassageTitle', 'Heading2', fontName='NanumGothicBold', fontSize=11, spaceAfter=6, textColor=colors.black)

    add_style('PassageBox', 'Normal', fontSize=9.5, leading=16, alignment=TA_JUSTIFY, backColor=colors.Color(0.97, 0.97, 0.97), borderWidth=0.5, borderColor=colors.gray, borderPadding=8, spaceAfter=10)

    add_style('QuestionTitle', 'Normal', fontName='NanumGothicBold', fontSize=10, spaceAfter=4, leading=13)
    add_style('ContextBox', 'Normal', fontSize=9.5, leading=14, alignment=TA_LEFT, backColor=colors.white, borderWidth=0.5, borderColor=colors.black, borderPadding=5, leftIndent=2, rightIndent=2, spaceAfter=4)
    add_style('OptionText', 'Normal', fontSize=9.5, leading=13, leftIndent=12, spaceAfter=2)
    add_style('AnswerTitle', 'Heading2', fontName='NanumGothicBold', fontSize=14, spaceBefore=15, spaceAfter=10, textColor=colors.darkblue)
    add_style('ExplanationBox', 'Normal', fontSize=9.5, leading=14, spaceAfter=10)

    return s

def clean_opt(text, idx):
    text = re.sub(r'^[①-⑤❶-❺1-5]\.?\s*|^\(\d+\)\s*', '', str(text))
    return f"({idx+1}) {text}"

def safe_sort_key(q):
    try:
        val = str(q.get('id', '0'))
        if '-' in val: return int(val.split('-')[1])
        return int(val)
    except: return 0

def format_passage(text):
    if not text: return ""
    text = text.replace('\n\n', '[[PARA]]')
    text = text.replace('\n', '[[PARA]]')
    text = text.replace('[[PARA]]', '<br/>&nbsp;')
    return text

# -----------------------------------------------------------
# [3] 데이터 로딩
# -----------------------------------------------------------
def balance_brackets(s):
    stack = []
    in_str = False
    escape = False
    for char in s:
        if escape: escape = False; continue
        if char == '"': in_str = not in_str
        elif not in_str:
            if char == '{': stack.append('}')
            elif char == '[': stack.append(']')
            elif char == '}' or char == ']':
                if stack and stack[-1] == char: stack.pop()
    return s + "".join(reversed(stack))

def load_data_safely(filepath):
    if not os.path.exists(filepath): return None
    with open(filepath, 'r', encoding='utf-8') as f:
        raw = f.read()
    content = re.sub(r'^```json\s*|\s*```$', '', raw).strip()
    content = re.sub(r'\[cite.*?\]', '', content)
    try: return json.loads(content)
    except Exception as e:
        print(f"❌ 데이터 해석 실패: {e}")
        return None
    try: return ast.literal_eval(balance_brackets(content))
    except: return None


# -----------------------------------------------------------
# [4] PDF 생성 메인 로직
# -----------------------------------------------------------
def create_korean_pdf():
    data = load_data_safely(INPUT_FILE)
    if not data: return

    doc = BaseDocTemplate(OUTPUT_FILE, pagesize=A4, margin=15*mm)
    page_w, page_h = A4
    margin, gap = 15*mm, 8*mm
    col_w = (page_w - 2*margin - gap) / 2
    full_width = page_w - 2*margin

    # [수정] 헤더 높이 축소 (40mm -> 30mm)
    header_height = 30*mm
    # [수정] 헤더와 본문 사이 간격 축소 (5mm -> 2mm)
    header_gap = 2*mm

    frames_first = [
        Frame(margin, page_h - margin - header_height, full_width, header_height, id='header', showBoundary=0),
        Frame(margin, margin, col_w, page_h - 2*margin - header_height - header_gap, id='col1', showBoundary=0),
        Frame(margin+col_w+gap, margin, col_w, page_h - 2*margin - header_height - header_gap, id='col2', showBoundary=0)
    ]

    frames_later = [
        Frame(margin, margin, col_w, page_h-2*margin, id='col1_full', showBoundary=0),
        Frame(margin+col_w+gap, margin, col_w, page_h-2*margin, id='col2_full', showBoundary=0)
    ]

    doc.addPageTemplates([
        PageTemplate(id='FirstPage', frames=frames_first),
        PageTemplate(id='LaterPage', frames=frames_later)
    ])

    story, s = [], get_custom_styles()
    info = data.get('test_info', {})

    # [1] 시험지 생성
    story.append(NextPageTemplate('LaterPage'))

    # 헤더
    story.append(Paragraph(info.get('title', '주간 국어 독해 평가'), s['MainTitle']))
    header_data = [[f"날짜: 2026. __. __", f"이름: ______________", f"점수: ______ / 100"]]
    h_tbl = Table(header_data, colWidths=[full_width*0.3, full_width*0.4, full_width*0.3])
    h_tbl.setStyle(TableStyle([
        ('FONTNAME', (0,0), (-1,-1), 'NanumGothic'),
        ('FONTSIZE', (0,0), (-1,-1), 10),
        ('ALIGN', (0,0), (-1,-1), 'CENTER'),
        ('LINEBELOW', (0,0), (-1,-1), 0.5, colors.black),
        ('BOTTOMPADDING', (0,0), (-1,-1), 8)
    ]))
    story.append(h_tbl)

    story.append(FrameBreak())

    all_qs_flat = []

    for idx, p in enumerate(data.get('passages', [])):
        if idx > 0:
            story.extend([Spacer(1, 4*mm), HRFlowable(width="100%", thickness=0.5, color=colors.black, dash=[2,2]), Spacer(1, 4*mm)])

        story.append(Paragraph(f"<b>[{idx+1}] {p.get('topic','주제 미정')}</b>", s['PassageTitle']))

        raw_content = p.get('passage_content', '')
        clean_content = format_passage(raw_content)
        story.append(Paragraph(clean_content, s['PassageBox']))

        story.append(Spacer(1, 2*mm))

        qs = p.get('questions', {})
        q_list = []
        if 'vocabulary' in qs: q_list.extend(qs['vocabulary'])
        if 'logic_ox' in qs: q_list.extend(qs['logic_ox'])
        if 'reading_comprehension' in qs: q_list.extend(qs['reading_comprehension'])
        if 'writing_strategy' in qs: q_list.extend(qs['writing_strategy'])
        if 'summary_cloze' in qs: q_list.extend(qs['summary_cloze'])
        q_list.sort(key=safe_sort_key)

        for q in q_list:
            q_id = q.get('id', '?')
            q_type = q.get('type', '')

            block = []

            q_title = f"{q_id}. {q.get('question','')}"
            block.append(Paragraph(q_title, s['QuestionTitle']))

            context_text = q.get('context') or q.get('statement') or q.get('summary_text')
            if context_text:
                block.append(Spacer(1, 2*mm))
                block.append(Paragraph(context_text, s['ContextBox']))
                block.append(Spacer(1, 4*mm))

            if 'options' in q and q['options']:
                for i, opt in enumerate(q['options']):
                    block.append(Paragraph(clean_opt(opt, i), s['OptionText']))

            if 'ox' in q_type or 'true_false' in q_type:
                block.append(Spacer(1, 2*mm))
                block.append(Paragraph("<b>답: ( O / X )</b> &nbsp;&nbsp;&nbsp; <b>이유:</b> " + "_"*20, s['OptionText']))

            block.append(Spacer(1, 5*mm))
            story.append(KeepTogether(block))
            all_qs_flat.append(q)

    # [2] 정답지 생성
    story.append(PageBreak())
    story.append(Paragraph("정답 및 해설(비문학독해)", s['MainTitle']))

    story.append(Paragraph("1. 빠른 정답 확인", s['AnswerTitle']))
    ans_data = [['문항', '정답'] * 2]

    chunk_size = (len(all_qs_flat) + 1) // 2
    for i in range(chunk_size):
        row = []
        if i < len(all_qs_flat):
            q = all_qs_flat[i]
            row.extend([q.get('id'), str(q.get('answer',''))])
        else:
            row.extend(['',''])
        if i + chunk_size < len(all_qs_flat):
            q = all_qs_flat[i + chunk_size]
            row.extend([q.get('id'), str(q.get('answer',''))])
        else:
            row.extend(['',''])
        ans_data.append(row)

    ans_tbl = Table(ans_data, colWidths=[15*mm, 20*mm, 15*mm, 20*mm], hAlign='LEFT')
    ans_tbl.setStyle(TableStyle([
        ('FONTNAME', (0,0), (-1,-1), 'NanumGothic'),
        ('GRID', (0,0), (-1,-1), 0.5, colors.grey),
        ('BACKGROUND', (0,0), (-1,0), colors.lightgrey),
        ('ALIGN', (0,0), (-1,-1), 'CENTER'),
        ('FONTSIZE', (0,0), (-1,-1), 9)
    ]))
    story.append(ans_tbl)
    story.append(Spacer(1, 8*mm))

    story.append(Paragraph("2. 문항별 상세 해설", s['AnswerTitle']))

    for q in all_qs_flat:
        q_id = q.get('id')
        ans = q.get('answer')
        expl = q.get('explanation', '해설 없음')

        story.append(Paragraph(f"<b>[{q_id}번] 정답: <font color='blue'>{ans}</font></b>", s['QuestionTitle']))
        story.append(Paragraph(f"<b>[해설]</b> {expl}", s['ExplanationBox']))
        story.append(HRFlowable(width="100%", thickness=0.2, color=colors.lightgrey, dash=[1,1], spaceAfter=4*mm))

    doc.build(story)
    print(f"\n✅ PDF 생성 완료! 파일 위치: {OUTPUT_FILE}")

if __name__ == "__main__":
    create_korean_pdf()

Mounted at /content/drive

✅ PDF 생성 완료! 파일 위치: /content/drive/MyDrive/weekly_kreading_test_data/Weekly_KReading_Review.pdf
